In [3]:
# numpy
import numpy as np
from numpy import unravel_index
print( 'Using numpy version %s' % np.__version__ )

# system
import glob, os

# astropy
import astropy as astro
from astropy.table import Table
from astropy.io import ascii, fits
from astropy.stats import sigma_clip
from astropy.convolution import convolve_fft
import astropy.units as u
from astropy.cosmology import Planck15
from astropy.coordinates import SkyCoord
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
print( 'Using astropy version %s' % astro.__version__ )

Using numpy version 1.25.2
Using astropy version 5.3.4


In [15]:
cat_f = ascii.read('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/Codes/filtered_data_table_goodss4.csv',format='csv')

In [16]:
print(cat_f)
print(cat_f.colnames)

  ID             IAU_designation            ...        tU_z       
----- ------------------------------------- ... ------------------
   18 CANDELS_GDS_F160W_J033224.84-275723.3 ...  2411234228.415048
   31 CANDELS_GDS_F160W_J033224.91-275714.4 ...  2402498254.289286
   40 CANDELS_GDS_F160W_J033221.18-275715.3 ...  5536057423.015836
   49 CANDELS_GDS_F160W_J033221.74-275708.7 ...  3210993224.622638
   73 CANDELS_GDS_F160W_J033223.88-275656.9 ...  4127161320.334082
   79 CANDELS_GDS_F160W_J033221.72-275701.9 ...  5704352551.862238
  107 CANDELS_GDS_F160W_J033220.42-275654.1 ...  7657686708.093629
  133 CANDELS_GDS_F160W_J033218.39-275639.8 ... 2130024561.7019992
  135 CANDELS_GDS_F160W_J033234.14-275640.0 ... 2268633729.4320045
  137 CANDELS_GDS_F160W_J033220.70-275642.3 ...  7232583625.799038
  ...                                   ... ...                ...
26707 CANDELS_GDS_F160W_J033241.55-274220.3 ...  2564899117.726414
26728 CANDELS_GDS_F160W_J033234.08-274222.2 ... 4244424049.881

In [17]:
cat_f['z'][0]

KeyError: 'z'

In [18]:
galaxy_IDS=np.unique(cat_f['CANDELS_ID'])
print(len(galaxy_IDS))

1491


# GOODSS- Cutouts

In [26]:
bands = ['275','435', '606', '775', '850', '125', '140', '160']

length = 200

for i in range( len( galaxy_IDS ) ):
    
    gal_id = galaxy_IDS[i]
    field = 'goodss'
    
    idx_in_catalog = np.where(cat_f['CANDELS_ID'] == gal_id)[0]

    if len(idx_in_catalog) > 1:
        coords = SkyCoord(cat_f['RA'][idx_in_catalog][0], cat_f['DEC'][idx_in_catalog][0], unit='deg')
    else:
        coords = SkyCoord(cat_f['RA'][idx_in_catalog], cat_f['DEC'][idx_in_catalog], unit='deg')
    
    if field == "goodss":
        
        for b in bands:
            # path to F275W mosaics
            if b == '275':
                band_ext = 'w'

                band_f_list = glob.glob('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/UV candels/*_goodss-*%s*.fits' % b)
                k_f = '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/kerneluv2h.fits'

                sci_ind = np.where( np.array( ['sci' in f for f in band_f_list] ) )[0][0]
                wht_ind = np.where( np.array( ['wht' in f for f in band_f_list] ) )[0][0]
                # exp_ind = np.where( np.array( ['exp' in f for f in band_f_list] ) )[0][0]
        
                # candels mosaic
                hdulist_c = fits.open(band_f_list[sci_ind])
                hdu_c = hdulist_c[0]
                wcs_c = WCS(hdu_c.header)
                wcs_c.sip = None

                # weight image
                hdulist_w = fits.open(band_f_list[wht_ind])
                hdu_w = hdulist_w[0]
                wcs_w = WCS(hdu_w.header)
                wcs_w.sip = None
                #hdulist_exp = fits.open(band_f_list[exp_ind])

                # psf-matching kernel
                hdulist_k = fits.open(k_f)
                hdu_k = hdulist_k[0]
                #hdu_exp = hdulist_exp[0]

                # create cutouts
                cutout_c = Cutout2D(hdu_c.data, coords, (length, length), wcs=wcs_c)
                cutout_w = Cutout2D(hdu_w.data, coords, (length, length), wcs=wcs_w)
                #cutout_exp = Cutout2D(hdu_exp.data, coords, (length, length), wcs=wcs_exp)

                cutout_c_conv = convolve_fft( cutout_c.data, hdu_k.data)
                #cutout_w_conv = convolve_fft( cutout_w.data, hdu_k.data)
                cutout_w = cutout_w.data
                #cutout_exp = cutout_exp.data

                img_wcs = cutout_c.wcs.to_header()
        
                convert_factor = hdu_c.header['PHOTFNU']
        
                hdu1 = fits.HDUList()
                hdu1.append(fits.PrimaryHDU(header=img_wcs))
                hdu1.append(fits.ImageHDU())
                hdu1.append(fits.ImageHDU())

                header = hdu1[0].header

                hdu1[0].data = cutout_c_conv
                hdu1[1].data = cutout_w

                header['Ext1'] = ('Science image')
                header['Ext2'] = ('Weight image')
                header['PHOT_ZP'] = (convert_factor, 'conversion factor to measure flux in Jy')

                hdu1.writeto('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/Cutouts_galaxies/goodss_%d_f%s%s_images.fits' % (gal_id, b, band_ext), overwrite=True)
                
                hdulist_c.close()
                #hdulist_exp.close()
                hdulist_k.close()
                hdulist_w.close()

            else:
                # list of mosaics
                band_f_list = glob.glob( '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_*%s*' % b )

                print(b, band_f_list)

                # If the bandpass!= F160W, retrieve the psf matching kernel.
                if b != '160':
                    kernel_f = glob.glob('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/psfs/goodss_*_psf/*%sW_kernel.fits.gz' % b)
                    band_ext = 'w'
        
                    # If the file naming convention is different:
                    if len( kernel_f ) == 0:
                        kernel_f = glob.glob('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/psfs/goodss_*_psf/*%sWcand_kernel.fits.gz' % b)
            
                        # For F850LP:
                        if len( kernel_f ) == 0:
                            kernel_f = glob.glob('//Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/psfs/goodss_*_psf/*%sLP_kernel.fits.gz' % b)
                            band_ext = 'lp'
                        
                    k_f = kernel_f[0]

            

                drz_ind = np.where( np.array( ['drz' in f for f in band_f_list] ) )[0][0]
                exp_ind = np.where( np.array( ['exp' in f for f in band_f_list] ) )[0][0]
                wht_ind = np.where( np.array( ['wht' in f for f in band_f_list] ) )[0][0]
        
                # candels mosaic
                hdulist_c = fits.open(band_f_list[drz_ind])
                hdu_c = hdulist_c[0]
                wcs_c = WCS(hdu_c.header)
                wcs_c.sip = None

                # weight image
                hdulist_w = fits.open(band_f_list[wht_ind])
                hdu_w = hdulist_w[0]
                wcs_w = WCS(hdu_w.header)
                wcs_w.sip = None

                # exposure time image
                hdulist_exp = fits.open(band_f_list[exp_ind])
                hdu_exp = hdulist_exp[0]
                wcs_exp = WCS(hdu_exp.header)
                wcs_exp.sip = None

                # 3dhst psf-matching kernel
                hdulist_k = fits.open(k_f)
                hdu_k = hdulist_k[0]

                # create cutouts
                cutout_c = Cutout2D(hdu_c.data, coords, (length, length), wcs=wcs_c)
                cutout_w = Cutout2D(hdu_w.data, coords, (length, length), wcs=wcs_w)
                cutout_exp = Cutout2D(hdu_exp.data, coords, (length, length), wcs=wcs_exp)
        
                if b != '160':
                    cutout_c_conv = convolve_fft( cutout_c.data, hdu_k.data)
                    #cutout_w_conv = convolve_fft( cutout_w.data, hdu_k.data)
                    cutout_w = cutout_w.data
                    cutout_exp = cutout_exp.data
                else:
                    cutout_c_conv = cutout_c.data
                    cutout_w = cutout_w.data
                    cutout_exp = cutout_exp.data

                img_wcs = cutout_c.wcs.to_header()
        
                if 'b' == '125' or b == '140' or b == '160':
                    convert_factor = hdu_c.header['PHOTFNU']
                else:
                    convert_factor = hdu_c.header['PHOTFLAM'] * hdu_c.header['PHOTPLAM'] ** 2 * 1e23 / (2.99792458e18)
        
                hdu1 = fits.HDUList()
                hdu1.append(fits.PrimaryHDU(header=img_wcs))
                hdu1.append(fits.ImageHDU())
                hdu1.append(fits.ImageHDU())
                hdu1.append(fits.ImageHDU())

                header = hdu1[0].header

                hdu1[0].data = cutout_c_conv
                hdu1[1].data = cutout_w
                hdu1[2].data = cutout_exp

                header['Ext1'] = ('Science image')
                header['Ext2'] = ('Weight image')
                header['Ext3'] = ('Exposure time image')
                header['PHOT_ZP'] = (convert_factor, 'conversion factor to measure flux in Jy')

                hdu1.writeto('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/Cutouts_galaxies/goodss_%d_f%s%s_images.fits' % (gal_id, b, band_ext), overwrite=True)
                
                hdulist_c.close()
                hdulist_exp.close()
                hdulist_k.close()
                hdulist_w.close()

435 ['/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_wht.fits', '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_exp.fits', '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits']
606 ['/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_drz.fits', '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_exp.fits', '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_wht.fits']
775 ['/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_drz.fits', '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_exp.fits', '

In [14]:
idx = np.where(cat_f['CANDELS_ID'] == 18)[0]

print(cat_f['z'][idx])

 z  
----
2.94
2.94


In [30]:
glob.glob('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clumps/UV candels/*_goodss-*.fits')

['/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clumps/UV candels/hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_wht.fits',
 '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clumps/UV candels/hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_sci.fits',
 '/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clumps/UV candels/hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_rms.fits']